reference:
- https://www.kaggle.com/code/toshikazuwatanabe/connect4-make-submission-with-stable-baselines3
- https://www.kaggle.com/code/kubamaliszewski/connectx-ppo

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import torch as th
from torch import nn
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import load_results
from stable_baselines3.common.torch_layers import NatureCNN
from stable_baselines3.common.policies import ActorCriticPolicy, ActorCriticCnnPolicy

In [ ]:
LOG_DIR = os.path.join(os.getcwd(), 'log')	# トレーニングのログを保存するディレクトリ
os.makedirs(LOG_DIR, exist_ok=True)

MODEL_DIR = os.path.join(os.pardir, 'models')	# トレーニング済みモデルを保存するディレクトリ
os.makedirs(MODEL_DIR, exist_ok=True)

MODEL_PATH = os.path.join(MODEL_DIR, 'connectx_model')	# トレーニング済みモデルのパス

環境

In [ ]:
# 環境の作成
from environment import ConnectFourGym

training_env = ConnectFourGym(opponent='random')
training_env

In [ ]:
# ログを取得する
training_env = Monitor(training_env, LOG_DIR, allow_early_resets=True)
training_env

In [ ]:
# 「DummyVecEnv」は、OpenAI Gymの環境をベクトル化するための特殊なラッパーです。
# 通常、強化学習アルゴリズムは一度に1つの環境しか処理できませんが、これを使用することで
# 複数の環境を同時に実行することができます。これにより、学習プロセスが効率的になります。
training_env = DummyVecEnv([lambda: training_env])
training_env

In [ ]:
training_env.observation_space.sample()

モデル

In [ ]:
from agent import CustomCNN

In [ ]:
POLICY_KWARGS = {
    'features_extractor_class': CustomCNN,
    'activation_fn': nn.ReLU, # ポリシーの中間層における活性化関数をReLU関数に設定
    'net_arch':[
        64,                 # 共有層（Shared Layer）に関する層
        dict(
            pi=[32, 16],    # 方策（Policy）に関する層
            vf=[32, 16]     # 価値関数（Value Function）に関する層
            )
        ],
    'features_extractor_kwargs': dict(features_dim = 768)   # CNNの出力次元数を768に設定
}

In [ ]:
if os.path.exists(MODEL_PATH):
    print('Loading existing model...')
    agent = PPO.load(MODEL_PATH, env=training_env, verbose=0)
else:
    print('Training new model...')
    agent = PPO(
        policy='MlpPolicy',
        env=training_env,
        policy_kwargs=POLICY_KWARGS,
        verbose=0,
	)

print(agent.policy)